# Arguments

In [19]:
# vgg16-ft-pv-self-eb,ap,lm

num_of_epochs = 100
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

final_model_path= 'tomato-model.cpickle'

input_width = 224
input_height = 224
input_depth = 3


# Select training classes

In [20]:
class_paths_validation = ['../datasets/validation/___Late_blight', '../datasets/validation/___Target_Spot', '../datasets/validation/___healthy', '../datasets/validation/___Mosaic_Virus', '../datasets/validation/___Yellow_Leaf_Curl_Virus', '../datasets/validation/___Leaf_Mold', '../datasets/validation/___Bacterial_spot', '../datasets/validation/___Early_blight']

# Load dataset

In [21]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
image_paths = []
print("[INFO] loading images...")
for path in class_paths_validation:
    image_paths.extend(list(paths.list_images(path)))

# Get unique classnames
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]

# Initial image preprocessing
aap = AspectAwarePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(test_x,test_y)  = sdl.load(image_paths,verbose=500)
test_x = test_x.astype("float") / 255.0


# convert the labels from integers to vectors
test_y = LabelBinarizer().fit_transform(test_y)

argument = [test_x, test_y]


[INFO] loading images...


In [22]:
print(len(class_names))
print(class_names)

8
['___Bacterial_spot', '___Early_blight', '___Late_blight', '___Leaf_Mold', '___Mosaic_Virus', '___Target_Spot', '___Yellow_Leaf_Curl_Virus', '___healthy']


# Load model from disk

In [23]:
import joblib
model = joblib.load(final_model_path)
# model.predict(argument)

In [24]:
from sklearn.metrics import classification_report

preds = model.predict(argument)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

ValueError: could not broadcast input array from shape (318,224,224,3) into shape (318)